In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
#df1 = pd.read_csv("../unFound/articles1.csv")

In [3]:
#df1.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [4]:
#df1.iloc[0,2]

'House Republicans Fret About Winning Their Health Care Suit - The New York Times'

In [5]:
#len(df1.iloc[0,9])

5607

In [6]:
'''



## its not that great dataset
'''

In [2]:
df4 = pd.read_csv("../unFound/headlines_test/news_summary_more.csv")

In [3]:
df4.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [36]:
#df5 = pd.read_csv("../unFound/headlines_test/news_summary.csv",encoding='latin-1' )

In [6]:
#df5.head()

In [4]:
df4.shape

(98401, 2)

In [40]:
## DF4 and 5 will do better , lets take df4

In [5]:
df_final = df4['headlines']

In [6]:
df_final = pd.DataFrame(df_final)

In [7]:
df_final.head()

,headlines
0,upGrad learner switches to career in ML & Al w...
1,Delhi techie wins free food from Swiggy for on...
2,New Zealand end Rohit Sharma-led India's 12-ma...
3,Aegon life iTerm insurance plan helps customer...
4,"Have known Hirani for yrs, what if MeToo claim..."


In [8]:
df_final.to_pickle("df_train.pkl")

## Data cleaning steps
1. Remove punctuations
2. Remove special characters
3. Make text to lower cases
4. Remove numerical values
5. Tokenize text
6. Remove stopwords

In [12]:
def data_clean_phase1(text):
    '''
    Remove special characters, numbers, convert to lower case, remove numerical values.
    '''
    text = text.lower()
    text = re.sub(r'\W', ' ', text) ## remove special characters
    text = re.sub(r'\s+[a-z]\s+', ' ', text) ## remove single character
    text = re.sub(r'^[a-z]\s+', ' ',text) # remove single characters at start of the string
    text = re.sub(r'\d', ' ', text) # remove digits
    text = re.sub(r'\s+', ' ', text) # remove extra spaces generated by removal of extra spaces
    return text
    
    

In [13]:
## Take a look at cleaned text

In [14]:
df_final_1 = df_final['headlines'].apply(lambda x: data_clean_phase1(x)) 

In [15]:
df_final_1 = pd.DataFrame(df_final_1)

In [16]:
df_final_1.head()

,headlines
0,upgrad learner switches to career in ml al wit...
1,delhi techie wins free food from swiggy for on...
2,new zealand end rohit sharma led india match w...
3,aegon life iterm insurance plan helps customer...
4,have known hirani for yrs what if metoo claims...


#### We have missed expansions
Lets move on, if result is not satisfactory we will expand and use bigram and after that lemmatization

In [17]:
### Document term matrix, using count vectoriser

from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv = CountVectorizer(stop_words='english', max_features=2000, max_df=0.8, min_df=1)
data_cv = cv.fit_transform(df_final_1['headlines']).toarray()


In [19]:
data_cv.shape

(98401, 2000)

In [ ]:
## Now we have to make a TFIDF model

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

In [21]:
tf_idf_transform = TfidfTransformer()

In [22]:
data_Tf_Idf = tf_idf_transform.fit_transform(data_cv)

In [24]:
data_tfidf = data_Tf_Idf.toarray()

In [28]:
data_tfidf = pd.DataFrame(data_tfidf, columns=cv.get_feature_names())
data_tfidf.index = df_final_1.index

In [30]:
data_tfidf.head()

,aadhaar,aamir,aap,abhishek,abuse,access,accident,accidentally,account,accounts,...,young,youngest,youth,youtube,yr,yrs,yuvraj,zealand,zero,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.399603,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.42328,0.0,0.000000,0.0,0.0


In [31]:
## Lets pickle it for later use
data_tfidf.to_pickle("dtm_tfidf.pkl")
#data_cv.to_pickle('dtm_cv.pkl')

In [32]:
df_final_1.to_pickle("clean_data_test.pkl")

In [33]:
data_Count_V = pd.DataFrame(data_cv, columns=cv.get_feature_names()) 

In [35]:
data_Count_V.to_pickle('data_count_vect.pkl')

In [9]:
df1 = pd.read_csv('articles1.csv')

In [10]:
df1.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [11]:
df1['title_len'] = df1['title'].apply(len)

In [12]:
df1['content_len'] = df1['content'].apply(len)

In [14]:
df1.head(2)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content,title_len,content_len
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,80,5607
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",91,27834


In [16]:
df1[['title_len', 'content_len']].describe()

,title_len,content_len
count,50000.000000,50000.0000
mean,71.923000,3853.4537
std,22.084418,3875.9117
min,2.000000,1.0000
25%,55.000000,1682.0000
50%,72.000000,2853.0000
75%,87.000000,5045.0000
max,231.000000,149346.0000


In [17]:
df2 = pd.read_csv('articles2.csv')

In [19]:
df2.head(2)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,53293,73471,Patriots Day Is Best When It Digs Past the Her...,Atlantic,David Sims,2017-01-11,2017.0,1.0,NaN,"Patriots Day, Peter Berg’s new thriller that r..."
1,53294,73472,A Break in the Search for the Origin of Comple...,Atlantic,Ed Yong,2017-01-11,2017.0,1.0,NaN,"In Norse mythology, humans and our world were ..."


In [24]:
df2['title_len'] = df2['title'].apply(len)
df2['content_len'] = df2['content'].apply(len)

In [22]:
type(df2['title'][0])

str

In [23]:
df2.dropna(inplace=True)

In [25]:
df2[['title_len', 'content_len']].describe()

,title_len,content_len
count,33695.000000,33695.000000
mean,57.150438,3920.726725
std,15.373395,3562.655717
min,1.000000,2.000000
25%,48.000000,1784.000000
50%,58.000000,3197.000000
75%,67.000000,4806.000000
max,123.000000,73366.000000
